In [1]:
import numpy as np
from copy import deepcopy
import time
from pycromanager import Acquisition, Bridge, multi_d_acquisition_events

In [2]:
bridge = Bridge()
mmc = bridge.get_core()
mmStudio = bridge.get_studio()

C:\Users\labelfree\.conda\envs\acq\lib\site-packages\pycromanager\core.py:258: UserWarning: Version mistmatch between Java ZMQ server and Python client. 
Java ZMQ server version: 4.0.0
Python client expected version: 4.1.0
 To fix, update to BOTH latest pycromanager and latest micro-manager nightly build
  warnings.warn(


In [3]:
mm_pos_list = mmStudio.get_position_list_manager().get_position_list()
number_of_positions = mm_pos_list.get_number_of_positions()

xyz_position_list = [[mm_pos_list.get_position(i).get_x(),
                      mm_pos_list.get_position(i).get_y(),
                      mm_pos_list.get_position(i).get_z()] \
                    for i in range(number_of_positions)]

offset = [mm_pos_list.get_position(i).get('PFSOffset').get1_d_position() for i in range(number_of_positions)]

xyz_position_list = np.around(xyz_position_list, decimals=3).astype('double')
offset = np.around(offset, decimals=3).astype('double')

epi_positions = xyz_position_list[:3]
lf_epi_positions = xyz_position_list[3:]

# epi_positions = [xyz_position_list[0]]
# lf_epi_positions = [xyz_position_list[1]]

xyz_position_list, offset

(array([[-7667.12 , 13888.929,  4527.86 ],
        [-7276.85 , 14041.279,  4529.9  ],
        [-7036.99 , 13982.73 ,  4528.92 ],
        [-7415.289, 14841.279,  4528.72 ],
        [-6362.16 , 14139.86 ,  4529.9  ],
        [-5916.95 , 13973.529,  4528.84 ]]),
 array([7381., 8775., 6907., 8641., 7666., 7224.]))

In [4]:
epi_channel = {'group': 'Master Channel', 'config': 'Epi'}
epi_exposure = 50

lf_state0 = {'group': 'Master Channel', 'config': 'LF-State0'}
lf_exposure = 20

# relative z
z_start = -2.5
z_end = 2.5
z_step = 0.25
z_range = np.arange(z_start, z_end + z_step, z_step)

num_time_points  = 5
time_interval_s  = 60  # in seconds

In [5]:
events = []
num_epi_positions = np.shape(epi_positions)[0]

for t in range(num_time_points):
    
    # epi-only positions, z stack events are created by autofocus function
    for p_idx, p in enumerate(epi_positions):
        events.append(
            {
                'axes': {'time': t, 'position':p_idx, 'z':0},
                'min_start_time': t*time_interval_s,
                'x': p[0],
                'y': p[1],
                'z': p[2],
                'channel': epi_channel,
                'exposure': epi_exposure,
                'keep_shutter_open': False,
            })

    # epi and lf positions, z stack events are created by autofocus function
    for p_idx, p in enumerate(lf_epi_positions):
        
        # epi channel
        events.append(
            {
                'axes': {'time': t,'position':p_idx+num_epi_positions, 'z':0},
                'min_start_time': t*time_interval_s,
                'x': p[0],
                'y': p[1],
                'z': p[2],
                'channel': epi_channel,
                'exposure': epi_exposure,
                'keep_shutter_open': False
            })
        
        events.append(
            {
                'axes': {'time': t,'position':p_idx+num_epi_positions, 'z':0},
                'min_start_time': t*time_interval_s,
                'x': p[0],
                'y': p[1],
                'z': p[2],
                'channel': lf_state0,
                'exposure': lf_exposure,
                'keep_shutter_open': False
            })

In [6]:
num_events = len(events)

def autofocus_fn(event, bridge, event_queue):
    if not hasattr(autofocus_fn, 'ctr'):
        autofocus_fn.ctr = 0
    
    mmc = bridge.get_core()
    
    # set PFS offset
    x_ = event['x']
    y_ = event['y']
    offset_ = offset[np.all(xyz_position_list[:,:2] == [x_,y_], axis=1)][0]
    mmc.set_position('PFSOffset', offset_)
    
    # autofocus
    try:
        mmc.full_focus()
    except:
        mmc.set_relative_position(5) # try moving up
        try:
            mmc.full_focus()
        except:
            mmc.set_relative_position(-10) # try moving down
            mmc.full_focus()
    
    time.sleep(1.0)
    pos = np.around(mmc.get_position(), decimals=3)

    # queue events
    if autofocus_fn.ctr < num_events-1:
        # queue epi channel
        for z_idx, z in enumerate(pos+z_range):
            new_event = deepcopy(event)
            new_event['axes']['z'] = z_idx
            new_event['z'] = z
            event_queue.put(new_event)

        # queue lf channels in ZC order
        if events[autofocus_fn.ctr+1]['channel'] == lf_state0:
            for z_idx, z in enumerate(pos+z_range):
                for ch in [f'LF-State{i}' for i in range(5)]:
                    new_event = deepcopy(event)
                    new_event['axes']['z'] = z_idx
                    new_event['z'] = z
                    new_event['channel']['config'] = ch
                    new_event['exposure'] = lf_exposure
                    event_queue.put(new_event)
            autofocus_fn.ctr += 1 # skip the next channel in the list

    if autofocus_fn.ctr < num_events-1:
        # queue next axes
        event_queue.put(events[autofocus_fn.ctr+1])
        autofocus_fn.ctr += 1
    else:
        event_queue.put(None) # stop acquisition


def hook_fn(event, bridge, event_queue):
    if 'axes' in event.keys():
        if not hasattr(hook_fn, 'pos_index'):
            hook_fn.pos_index = event['axes']['position']
            autofocus_fn(event, bridge, event_queue)
            hook_fn.focused = True
            return
            
        if event['axes']['position'] != hook_fn.pos_index and not hook_fn.focused:
            autofocus_fn(event, bridge, event_queue)
            hook_fn.focused = True
            return
            
        hook_fn.pos_index = event['axes']['position']
        hook_fn.focused = False
    
    return event

In [7]:
mmc.set_auto_shutter(True)
# mmc.set_auto_focus_device('PFS')
# mmc.set_config('Epi Channel', 'GFP EX466-40 EM525-50')

snap_live_manager = mmStudio.get_snap_live_manager()
if snap_live_manager.is_live_mode_on():
    snap_live_manager.set_live_mode_on(False)

acq = Acquisition(directory=r'D:\2022_02_15 Beads', name='pfs_test', post_hardware_hook_fn=hook_fn)
acq.acquire(events[0])